We want to train a train a neural network which can reverse a sentence. 
We will first use an encoder/decoder without attention and then try with attention. 

Mostly trying to re-implement https://towardsdatascience.com/attention-seq2seq-with-pytorch-learning-to-invert-a-sequence-34faf4133e53

In [48]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import random
from collections import defaultdict

class ToyDataset(Dataset):
    """
    Generate a dataset of sentences and their reverse sentences of variable length.
    """
    def __init__(self, words=[], min_len=5, max_len=10, type='train'):
        self.min_len = min_len
        self.max_len = max_len
        # if no words are passed, let them be simple charecters
        if not words:
            self.words = list('abcdef')

        # start of sentence
        self.eos = '<eos>'
        self.sos = '<sos>'
        self.pad = '<pad>'

        # complete vocab
        self.vocab = [self.pad, self.sos, self.eos] + self.words

        self.w2i = {self.vocab[idx]:idx for idx in range(len(self.vocab))}
        self.i2w = {idx:self.vocab[idx] for idx in range(len(self.vocab))}

        if type == 'train':
            self.set = [self._sample() for _ in range(3000)]
        else:
            self.set = [self._sample() for _ in range(300)]

    def _sample(self):
        size = random.randint(self.min_len, self.max_len)

        # ignore the last two as they are sos and eos
        sentence = random.choices(self.words, k=size)
        reverse = [sentence[idx] for idx in range(len(sentence)-1, -1, -1)]
        
        # add the sos and eos
        sentence = [self.sos] + sentence + [self.eos]
        reverse = [self.sos] + reverse + [self.eos]
        
        # padding. Add 2 as we added sos and eos
        sentence += [self.pad] * (self.max_len + 2 - len(sentence))
        reverse += [self.pad] * (self.max_len + 2 - len(reverse))

    
        # let convert the words to integers
        sentence_tensor = torch.LongTensor([self.w2i[word] for word in sentence])
        reverse_tensor = torch.LongTensor([self.w2i[word] for word in reverse])
        return sentence_tensor, reverse_tensor

    def __len__(self):
        return len(self.set)

    def __getitem__(self, item):
        return self.set[item]

Lets see an sample

In [52]:
ds = ToyDataset()
idx = random.randint(0, len(ds))
ds[idx]

(tensor([1, 3, 7, 3, 3, 6, 6, 2, 0, 0, 0, 0]),
 tensor([1, 6, 6, 3, 3, 7, 3, 2, 0, 0, 0, 0]))

In [41]:
train_dl = DataLoader(ds, batch_size=3)
batch = next(iter(train_dl))

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 10 and 11 in dimension 1 at /Users/distiller/project/conda/conda-bld/pytorch_1579022036889/work/aten/src/TH/generic/THTensor.cpp:612

### Encoder

In [ ]:
"""
Stacked Linear Layers with Relu
"""
class LinearRelu(nn.Module):
    def __init__(self, input_size, hidden_sizes):
        super(LinearRelu, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.relu = nn.ReLU()
        
        self.layers = nn.ModuleList()
        for idx in range(len(hidden_sizes)):
            if idx == 0:
                self.layers.append(nn.Linear(input_size, hidden_sizes[idx]))
            else:
                self.layers.append(nn.Linear(hidden_sizes[idx-1], hidden_sizes[idx]))

    def forward(self, x):
        for idx, layer in enumerate(self.layers):
            print(idx, layer)
            x = self.relu(layer(x))
        return x

In [ ]:
"""
We will use a bidirectional LSTM to encode the input
"""
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_sizes,embed_size=10, vocab_size=):
        super(Encoder, self).__init__()
        self.embedd
        
    def 
        
    